# **Capstone Project - The Battle of Neighborhoods**

### Import Libraries

In this section we import the libraries that are required

In [11]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

from geopy.geocoders import Nominatim
import urllib.request
import json
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from urllib.request import urlopen

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as pyplot
from sklearn.cluster import KMeans

import folium
print('Libarries are imported')

Libarries are imported


### Download and understad Datset

Download and understand the dataset Neighborhood has total 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, essentially need a dataset the contains the 5 boroughs and the neighborhoods that exists in borough as week as the latitude and longitude coordinates of each neighborhood.
This dataset exists for free on the web, the link to the dataset : https://geo.nyu.edu/catalog/nyu_2451_34572 

In [17]:
import requests
fname = 'nyu_2451_34572-geojson.json'
url = 'https://cocl.us/new_york_dataset/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

with open('nyu_2451_34572-geojson.json') as json_data:
    ny_data = json.load(json_data)

**Transform data into dataframe**

In [19]:
neighborhoods_data = ny_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
										  
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


**Use geopy library to get the latitude and longitude values of New York City.**

In order to define an instance of the geocoder, Need to define a user_agent. Will name agent ny_explorer.


In [20]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


**Create a map of New York with neighborhoods superimposed on top.**

In [21]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [25]:
import folium
# create map of New York using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

**Foursquare Venues**

In [27]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [29]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'G3XZBQIPEG3WSYI4GHLQQROQCFU1KIP2J0AYN1L14ZYVPB5O'
CLIENT_SECRET = 'MKBQMOPBYDNKD3MJU0GDYLXWMMNWNRLOSEVV14GSMSVSKMDY'
VERSION = '20200830'
#https://developer.foursquare.com/docs/resources/categories
#Asian = 4bf58dd8d48988d142941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_asian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d142941735')
newyork_venues_asian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,China Wang,40.874347,-73.910540,Chinese Restaurant
1,Marble Hill,40.876551,-73.91066,Mei Chung Mei Restaurant,40.878272,-73.902981,Chinese Restaurant
2,Marble Hill,40.876551,-73.91066,Empire Chinese And Japanese Cuisine,40.878588,-73.917446,Japanese Restaurant
3,Marble Hill,40.876551,-73.91066,Foo Hing Restaurant,40.874984,-73.901575,Chinese Restaurant
4,Marble Hill,40.876551,-73.91066,Anise,40.885989,-73.910036,Chinese Restaurant


In [30]:
newyork_venues_asian.shape

(1934, 7)

In [32]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [33]:
map_newyork_asian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_asian, 'red', map_newyork_asian)

map_newyork_asian

In [34]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [35]:
manhattan_grouped = newyork_venues_asian.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,50,50,50,50,50,50
Central Harlem,49,49,49,49,49,49
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,49,49,49,49,49,49
East Village,50,50,50,50,50,50


**Analyze Each Neighborhood**

In [36]:
manhattan_onehot = pd.get_dummies(newyork_venues_asian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_asian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Bar,Beer Bar,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Gastropub,Grocery Store,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,New American Restaurant,Noodle House,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Smoothie Shop,Soba Restaurant,Street Food Gathering,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Bar,Beer Bar,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Gastropub,Grocery Store,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,New American Restaurant,Noodle House,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Smoothie Shop,Soba Restaurant,Street Food Gathering,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Battery Park City,0.00,0.040000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.260000,0.000000,0.00,0.00,0.02,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.06,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.160000,0.040000,0.000000,0.00,0.020000,0.00,0.000000,0.060000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.180000,0.000000,0.00,0.00,0.040000,0.02,0.00,0.00,0.00,0.00,0.040000,0.000000,0.02
1,Carnegie Hill,0.00,0.040000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.140000,0.000000,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.140000,0.020000,0.000000,0.00,0.020000,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.240000,0.020000,0.00,0.00,0.240000,0.00,0.00,0.00,0.00,0.02,0.060000,0.000000,0.00
2,Central Harlem,0.00,0.061224,0.00,0.00,0.00,0.00,0.00,0.020408,0.00,0.00,0.551020,0.020408,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020408,0.081633,0.000000,0.000000,0.00,0.020408,0.00,0.000000,0.020408,0.00,0.00,0.00,0.020408,0.00,0.00,0.00,0.00,0.122449,0.000000,0.00,0.00,0.061224,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00
3,Chelsea,0.00,0.060000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.120000,0.020000,0.00,0.02,0.00,0.00,0.02,0.000000,0.02,0.02,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.120000,0.100000,0.040000,0.00,0.060000,0.00,0.000000,0.080000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.200000,0.000000,0.00,0.00,0.080000,0.00,0.00,0.00,0.02,0.00,0.020000,0.000000,0.00
4,Chinatown,0.00,0.000000,0.04,0.00,0.00,0.02,0.00,0.000000,0.04,0.02,0.240000,0.000000,0.00,0.00,0.00,0.00,0.10,0.040000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.040000,0.020000,0.060000,0.00,0.040000,0.02,0.000000,0.020000,0.00,0.00,0.02,0.000000,0.02,0.00,0.00,0.00,0.040000,0.000000,0.02,0.00,0.080000,0.00,0.00,0.00,0.00,0.04,0.060000,0.000000,0.00
5,Civic Center,0.00,0.020000,0.04,0.00,0.00,0.02,0.00,0.000000,0.02,0.02,0.280000,0.000000,0.00,0.00,0.00,0.00,0.08,0.020000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.040000,0.000000,0.040000,0.00,0.080000,0.02,0.000000,0.000000,0.00,0.00,0.02,0.020000,0.02,0.00,0.00,0.00,0.060000,0.000000,0.04,0.00,0.060000,0.00,0.00,0.00,0.00,0.04,0.040000,0.000000,0.00
6,Clinton,0.02,0.020000,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.120000,0.000000,0.02,0.02,0.00,0.00,0.02,0.000000,0.02,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.120000,0.080000,0.000000,0.00,0.020000,0.00,0.000000,0.180000,0.00,0.02,0.00,0.000000,0.00,0.00,0.00,0.00,0.100000,0.000000,0.00,0.00,0.180000,0.00,0.00,0.00,0.00,0.00,0.020000,

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Food Truck,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Korean Restaurant,Deli / Bodega
1,Carnegie Hill,Thai Restaurant,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Noodle House
2,Central Harlem,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Café,Noodle House,Cocktail Bar,Ramen Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Korean Restaurant,Thai Restaurant,Ramen Restaurant,Asian Restaurant,Noodle House,Malay Restaurant,Fish Market
4,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Malay Restaurant,Vietnamese Restaurant,Sushi Restaurant,Japanese Restaurant,Noodle House,Cantonese Restaurant,Dumpling Restaurant


**Cluster Neighborhoods**

In [40]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 2, 4, 3, 3, 4, 2, 4, 3])

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Korean Restaurant,Sushi Restaurant,Xinjiang Restaurant,Food Court,Filipino Restaurant,Fish Market
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Malay Restaurant,Vietnamese Restaurant,Sushi Restaurant,Japanese Restaurant,Noodle House,Cantonese Restaurant,Dumpling Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant,Japanese Restaurant,Xinjiang Restaurant,Food & Drink Shop,Filipino Restaurant,Fish Market
3,Manhattan,Inwood,40.867684,-73.921210,2,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,Asian Restaurant,Dumpling Restaurant,Xinjiang Restaurant,Food Court,Filipino Restaurant,Fish Market,Food & Drink Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Cocktail Bar,Noodle House,Xinjiang Restaurant,Filipino Restaurant


In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Asian Restaurant,Ramen Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Dim Sum Restaurant
9,Yorkville,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Ramen Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Dim Sum Restaurant
10,Lenox Hill,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Japanese Restaurant,Asian Restaurant,Noodle House,Dim Sum Restaurant,Beer Bar,Ramen Restaurant,Szechuan Restaurant
11,Roosevelt Island,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant,Korean Restaurant,Noodle House,Beer Bar,Cocktail Bar,Ramen Restaurant
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Asian Restaurant,Ramen Restaurant,Vietnamese Restaurant,Peruvian Restaurant,Dim Sum Restaurant,Grocery Store
30,Carnegie Hill,Thai Restaurant,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Noodle House


In [44]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Korean Restaurant,Japanese Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Ramen Restaurant,Cuban Restaurant,Dim Sum Restaurant,Coffee Shop,Filipino Restaurant
16,Murray Hill,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Szechuan Restaurant,Ramen Restaurant,Salad Place,Chinese Restaurant,Cuban Restaurant
27,Gramercy,Korean Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Tibetan Restaurant,Vietnamese Restaurant,Udon Restaurant,Chinese Restaurant,Bar
33,Midtown South,Korean Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Cuban Restaurant,Ramen Restaurant,Chinese Restaurant,Szechuan Restaurant,Salad Place
36,Tudor City,Korean Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Udon Restaurant
38,Flatiron,Korean Restaurant,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Ramen Restaurant,Food Court,Fried Chicken Joint


In [45]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Korean Restaurant,Sushi Restaurant,Xinjiang Restaurant,Food Court,Filipino Restaurant,Fish Market
2,Washington Heights,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant,Japanese Restaurant,Xinjiang Restaurant,Food & Drink Shop,Filipino Restaurant,Fish Market
3,Inwood,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,Asian Restaurant,Dumpling Restaurant,Xinjiang Restaurant,Food Court,Filipino Restaurant,Fish Market,Food & Drink Shop
4,Hamilton Heights,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Cocktail Bar,Noodle House,Xinjiang Restaurant,Filipino Restaurant
5,Manhattanville,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant,Japanese Curry Restaurant,Ramen Restaurant,Noodle House,Cocktail Bar,Korean Restaurant
6,Central Harlem,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Café,Noodle House,Cocktail Bar,Ramen Restaurant
7,East Harlem,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Noodle House,Asian Restaurant,Hawaiian Restaurant,Japanese Restaurant,Xinjiang Restaurant,Filipino Restaurant,Fish Market


In [46]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Malay Restaurant,Vietnamese Restaurant,Sushi Restaurant,Japanese Restaurant,Noodle House,Cantonese Restaurant,Dumpling Restaurant
21,Tribeca,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Vietnamese Restaurant,Bakery,Thai Restaurant,Noodle House,Sandwich Place,Asian Restaurant,Cantonese Restaurant
22,Little Italy,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Sushi Restaurant,Malay Restaurant,Japanese Restaurant,Cantonese Restaurant,Noodle House,Bakery,Peking Duck Restaurant
23,Soho,Chinese Restaurant,Dim Sum Restaurant,Sushi Restaurant,Vietnamese Restaurant,Noodle House,Thai Restaurant,Japanese Restaurant,Malay Restaurant,Vegetarian / Vegan Restaurant,Bakery
25,Manhattan Valley,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Noodle House,Szechuan Restaurant,Vietnamese Restaurant,Malay Restaurant,Cuban Restaurant,Korean Restaurant
26,Morningside Heights,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Vietnamese Restaurant,Japanese Restaurant,Szechuan Restaurant,Korean Restaurant,Peruvian Restaurant,Ramen Restaurant
28,Battery Park City,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Food Truck,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Korean Restaurant,Deli / Bodega
29,Financial District,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Food Truck,Thai Restaurant,Xinjiang Restaurant,Seafood Restaurant,Korean Restaurant,Street Food Gathering
32,Civic Center,Chinese Restaurant,Noodle House,Dim Sum Restaurant,Thai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Japanese Restaurant
34,Sutton Place,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Vietnamese Restaurant,Peking Duck Restaurant,Deli / Bodega


In [47]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Korean Restaurant,Chinese Restaurant,Vietnamese Restaurant,Asian Restaurant,Coffee Shop,Fried Chicken Joint
14,Clinton,Thai Restaurant,Ramen Restaurant,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,American Restaurant,Japanese Curry Restaurant,Filipino Restaurant,Noodle House
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Korean Restaurant,Thai Restaurant,Ramen Restaurant,Asian Restaurant,Noodle House,Malay Restaurant,Fish Market
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Vietnamese Restaurant,Udon Restaurant,Korean Restaurant,Noodle House,Dumpling Restaurant,Indonesian Restaurant
19,East Village,Sushi Restaurant,Vietnamese Restaurant,Korean Restaurant,Japanese Restaurant,Thai Restaurant,Dumpling Restaurant,Chinese Restaurant,Ramen Restaurant,Taiwanese Restaurant,Tibetan Restaurant
20,Lower East Side,Vietnamese Restaurant,Sushi Restaurant,Chinese Restaurant,Thai Restaurant,Dumpling Restaurant,Korean Restaurant,Tibetan Restaurant,Dim Sum Restaurant,Ramen Restaurant,Sake Bar
24,West Village,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Ramen Restaurant,Korean Restaurant,Vietnamese Restaurant,Noodle House,Asian Restaurant,Cocktail Bar
31,Noho,Chinese Restaurant,Vietnamese Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Dumpling Restaurant,Korean Restaurant,Taiwanese Restaurant,Ramen Restaurant,Dim Sum Restaurant
35,Turtle Bay,Japanese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Asian Restaurant,Ramen Restaurant,Thai Restaurant,Chinese Restaurant,Peking Duck Restaurant,Korean Restaurant,Shanghai Restaurant
37,Stuyvesant Town,Sushi Restaurant,Ramen Restaurant,Vietnamese Restaurant,Thai Restaurant,Korean Restaurant,Taiwanese Restaurant,Chinese Restaurant,Japanese Restaurant,Asian Restaurant,Tibetan Restaurant
